In [19]:
cd("/Users/jerobinett/Desktop/ERA5Analysis/Basin Data")
burrowactivate()
import ERA5Analysis as ERA
using PyCall, Dictionaries, DataFrames, Shapefile
@pyimport matplotlib.pyplot as plt
@pyimport matplotlib.patches as mpatches
@pyimport cartopy.crs as ccrs

  Activating project at `~/Desktop/ERA5Analysis`


In [20]:

#use for allowed ids

huc_paths = "HUC_Shapes/WBDHU".*string.([6,8]).*".shp"
shapes = Dictionary([6,8],DataFrame.(Shapefile.Table.(huc_paths)))

ax = plt.subplot(1,1,1,projection = ccrs.AlbersEqualArea(central_longitude = -150,
            standard_parallels = (57,69)))

basincolors = Dictionary(ERA.basin_names,["red","green","purple","orange","blue", "pink", "yellow","black"])

for (basin_ids, basin_name) in zip(ERA.allowed_hucs, ERA.basin_names)
    for huc in basin_ids
        shapefile = shapes[length(huc)]
        basin_idx = findfirst( ==(huc), shapefile[!, "huc$(length(huc))"])
        #and use that to grab the polygon
        my_polygon = shapefile[basin_idx, :geometry]
        poly_verts = hcat(getproperty.(my_polygon.points, :x), getproperty.(my_polygon.points, :y))
        poly=mpatches.Polygon(poly_verts, closed=true, alpha=0.7, facecolor = basincolors[basin_name],
        transform = ccrs.PlateCarree())
        ax.add_patch(poly)
    end
end
ax.set_extent(ERA.ak_bounds, crs=ccrs.PlateCarree())
ax.gridlines()
ax.coastlines()
plt.savefig("basin_vis.png")